In [1]:
%load_ext autoreload
%autoreload 2

%cd ../

/Users/macos/Uni/1st_year/period_3/DSProj/code/models


In [2]:
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy
import seaborn as sns

import evaluation
import utils

In [3]:
plt.style.use('seaborn-v0_8')
plt.rcParams.update({'font.size': 8})

In [4]:
PATH_DIR_DATA_PROCESS = Path("data_processed")

## 0. Load data and encoding

In [5]:
path_dir_embd = Path("data_processed/mf_PROBABILITY_OUTPUT=True/Mar04_15-20-31")

path_embd_site = path_dir_embd / "emb_sites.npy"
path_embd_species = path_dir_embd / "emb_species.npy"

emb_site = np.load(path_embd_site).squeeze()
emb_species = np.load(path_embd_species).squeeze()

In [6]:
path_dir_encode = PATH_DIR_DATA_PROCESS / "encoder"

path_enc_genera = path_dir_encode / "ordinal_enc_species.json"
path_enc_site = path_dir_encode / "ordinal_enc_site.json"

enc_genera = utils.CategoryDict.from_file(path_enc_genera)
enc_site = utils.CategoryDict.from_file(path_enc_site)

In [7]:
data_train = np.load(PATH_DIR_DATA_PROCESS / "trainval/data_train.npy", allow_pickle=True)
data_val = np.load(PATH_DIR_DATA_PROCESS / "trainval/data_val.npy", allow_pickle=True)

df_train = utils.conv_dataset_patch2df(data_train)
df_val = utils.conv_dataset_patch2df(data_val)

print(f"train: {df_train.shape}")
print(f"val: {df_val.shape}")

df_val.head()

train: (304000, 3)
val: (20536, 3)


,site,species,occurence
0,198,116,0.0
1,198,117,0.0
2,198,118,0.0
3,198,119,0.0
4,199,116,0.0


## 2. Start validating

In [8]:
N = 15
N_SITES = 10

In [9]:
emb_species_normed = emb_species / np.clip(np.linalg.norm(emb_species, axis=1)[:, None], a_max=10, a_min=1e-6)

In [10]:
for species in df_val['species'].unique():
    df_val_species = df_val[df_val['species'] == species]

    emb = emb_species[species]
    emb = emb / np.clip(np.linalg.norm(emb), a_min=1e-6, a_max=10)
    sim = emb_species_normed @ emb[:, None]

    # Get top N similar species
    idx_top = np.argpartition(sim.squeeze(), -N)[-N:]

    sites_occ_pred = df_train[
        (df_train['species'].isin(idx_top)) & 
        (df_train['site'].isin(df_val_species['site']))
    ]\
        .groupby(by='site')\
        .mean()['occurence']

    df_val.loc[df_val_species.sort_values(by='site').index, 'pred'] = sites_occ_pred.values

In [12]:
print(f"TPR : {evaluation.calc_tpr(df_val):.6f}")
print(f"Expected Percentile Ranking : {evaluation.calc_expected_percentile_rank(df_val):.6f}")

TPR : 0.003896
Expected Percentile Ranking : 0.236931
